In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten, Conv2D, MaxPooling2D
import warnings
warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ghassenabdedayem/AltCoins/master/data_sources/df_global_0.csv', sep = ',')
df.head()

,GOLD_Time,GOLD_USD_AM,GOLD_USD_PM,GOLD_GBP_AM,GOLD_GBP_PM,GOLD_EUR_AM,GOLD_EUR_PM,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap,ADA_Open,ADA_High,ADA_Low,ADA_Close,ADA_Volume,ADA_Market_Cap,LTC_Open,LTC_High,LTC_Low,LTC_Close,LTC_Volume,LTC_Market_Cap,EOS_Open,EOS_High,EOS_Low,EOS_Close,EOS_Volume,EOS_Market_Cap,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Volume,ETH_Market_Cap,BCH_Open,BCH_High,BCH_Low,...,ICE_Low,ICE_Close,ICE_Volume,IEF_Open,IEF_High,IEF_Low,IEF_Close,IEF_Volume,NVIDIA_Open,NVIDIA_High,NVIDIA_Low,NVIDIA_Close,NVIDIA_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Volume,GOOGLE_Open,GOOGLE_High,GOOGLE_Low,GOOGLE_Close,GOOGLE_Volume,GPS_Open,GPS_High,GPS_Low,GPS_Close,GPS_Volume,COCOA_Open,COCOA_High,COCOA_Low,COCOA_Close,COCOA_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Volume,Day_Cos,Day_Sin
0,2019-12-31,1523.0,1523.0,1157.78,1157.78,1358.06,1358.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,230.50,235.675,230.20,235.30,5766007.0,45.86,46.09,44.927,45.86,31562358.0,1330.11,1338.00,1329.085,1337.02,961438.0,17.76,17.860,17.58,17.69,0.0,NaN,NaN,NaN,NaN,0.0,132.55,134.110,132.44,133.98,0.0,-0.222521,9.749279e-01
1,2019-12-30,1511.5,1515.0,1152.37,1152.57,1350.22,1351.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,235.84,236.200,231.09,232.32,6263092.0,45.52,46.17,44.660,45.52,40899781.0,1350.00,1353.00,1334.020,1336.14,1047066.0,17.81,18.070,17.62,17.81,0.0,NaN,NaN,NaN,NaN,0.0,135.25,135.278,132.50,132.78,0.0,0.623490,7.818315e-01
2,2019-12-29,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.000000,-2.449294e-16
3,2019-12-28,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.623490,-7.818315e-01
4,2019-12-27,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,239.61,240.340,235.81,236.87,6320454.0,46.85,46.88,45.975,46.18,36324219.0,1362.99,1364.24,1349.310,1351.89,1030643.0,17.99,18.039,17.69,17.84,0.0,NaN,NaN,NaN,NaN,0.0,135.00,135.750,134.87,135.26,0.0,-0.222521,-9.749279e-01


In [0]:
df = df.set_index('GOLD_Time')
df.head()

,GOLD_USD_AM,GOLD_USD_PM,GOLD_GBP_AM,GOLD_GBP_PM,GOLD_EUR_AM,GOLD_EUR_PM,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap,ADA_Open,ADA_High,ADA_Low,ADA_Close,ADA_Volume,ADA_Market_Cap,LTC_Open,LTC_High,LTC_Low,LTC_Close,LTC_Volume,LTC_Market_Cap,EOS_Open,EOS_High,EOS_Low,EOS_Close,EOS_Volume,EOS_Market_Cap,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Volume,ETH_Market_Cap,BCH_Open,BCH_High,BCH_Low,BCH_Close,...,ICE_Low,ICE_Close,ICE_Volume,IEF_Open,IEF_High,IEF_Low,IEF_Close,IEF_Volume,NVIDIA_Open,NVIDIA_High,NVIDIA_Low,NVIDIA_Close,NVIDIA_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Volume,GOOGLE_Open,GOOGLE_High,GOOGLE_Low,GOOGLE_Close,GOOGLE_Volume,GPS_Open,GPS_High,GPS_Low,GPS_Close,GPS_Volume,COCOA_Open,COCOA_High,COCOA_Low,COCOA_Close,COCOA_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Volume,Day_Cos,Day_Sin
GOLD_Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,1523.0,1523.0,1157.78,1157.78,1358.06,1358.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,230.50,235.675,230.20,235.30,5766007.0,45.86,46.09,44.927,45.86,31562358.0,1330.11,1338.00,1329.085,1337.02,961438.0,17.76,17.860,17.58,17.69,0.0,NaN,NaN,NaN,NaN,0.0,132.55,134.110,132.44,133.98,0.0,-0.222521,9.749279e-01
2019-12-30,1511.5,1515.0,1152.37,1152.57,1350.22,1351.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,235.84,236.200,231.09,232.32,6263092.0,45.52,46.17,44.660,45.52,40899781.0,1350.00,1353.00,1334.020,1336.14,1047066.0,17.81,18.070,17.62,17.81,0.0,NaN,NaN,NaN,NaN,0.0,135.25,135.278,132.50,132.78,0.0,0.623490,7.818315e-01
2019-12-29,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.000000,-2.449294e-16
2019-12-28,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.623490,-7.818315e-01
2019-12-27,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,239.61,240.340,235.81,236.87,6320454.0,46.85,46.88,45.975,46.18,36324219.0,1362.99,1364.24,1349.310,1351.89,1030643.0,17.99,18.039,17.69,17.84,0.0,NaN,NaN,NaN,NaN,0.0,135.00,135.750,134.87,135.26,0.0,-0.222521,-9.749279e-01


On va remplacer les valeurs par la variation par rapport à la valeur précédente en % :

In [0]:
for col in df.columns:
  for i in range(len(df)):
    if np.isnan(pd.to_numeric(df[col][i])):
      df[col][i] = 0

df.head()

,GOLD_USD_AM,GOLD_USD_PM,GOLD_GBP_AM,GOLD_GBP_PM,GOLD_EUR_AM,GOLD_EUR_PM,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap,ADA_Open,ADA_High,ADA_Low,ADA_Close,ADA_Volume,ADA_Market_Cap,LTC_Open,LTC_High,LTC_Low,LTC_Close,LTC_Volume,LTC_Market_Cap,EOS_Open,EOS_High,EOS_Low,EOS_Close,EOS_Volume,EOS_Market_Cap,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Volume,ETH_Market_Cap,BCH_Open,BCH_High,BCH_Low,BCH_Close,...,ICE_Low,ICE_Close,ICE_Volume,IEF_Open,IEF_High,IEF_Low,IEF_Close,IEF_Volume,NVIDIA_Open,NVIDIA_High,NVIDIA_Low,NVIDIA_Close,NVIDIA_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Volume,GOOGLE_Open,GOOGLE_High,GOOGLE_Low,GOOGLE_Close,GOOGLE_Volume,GPS_Open,GPS_High,GPS_Low,GPS_Close,GPS_Volume,COCOA_Open,COCOA_High,COCOA_Low,COCOA_Close,COCOA_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Volume,Day_Cos,Day_Sin
GOLD_Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,1523.0,1523.0,1157.78,1157.78,1358.06,1358.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,230.50,235.675,230.20,235.30,5766007.0,45.86,46.09,44.927,45.86,31562358.0,1330.11,1338.00,1329.085,1337.02,961438.0,17.76,17.860,17.58,17.69,0.0,0.0,0.0,0.0,0.0,0.0,132.55,134.110,132.44,133.98,0.0,-0.222521,9.749279e-01
2019-12-30,1511.5,1515.0,1152.37,1152.57,1350.22,1351.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,235.84,236.200,231.09,232.32,6263092.0,45.52,46.17,44.660,45.52,40899781.0,1350.00,1353.00,1334.020,1336.14,1047066.0,17.81,18.070,17.62,17.81,0.0,0.0,0.0,0.0,0.0,0.0,135.25,135.278,132.50,132.78,0.0,0.623490,7.818315e-01
2019-12-29,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,0.0,0.00,0.00,0.000,0.00,0.0,0.00,0.00,0.000,0.00,0.0,0.00,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,0.0,1.000000,-2.449294e-16
2019-12-28,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,0.0,0.00,0.00,0.000,0.00,0.0,0.00,0.00,0.000,0.00,0.0,0.00,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,0.0,0.623490,-7.818315e-01
2019-12-27,1510.6,1511.5,1156.36,1152.67,1356.37,1353.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,239.61,240.340,235.81,236.87,6320454.0,46.85,46.88,45.975,46.18,36324219.0,1362.99,1364.24,1349.310,1351.89,1030643.0,17.99,18.039,17.69,17.84,0.0,0.0,0.0,0.0,0.0,0.0,135.00,135.750,134.87,135.26,0.0,-0.222521,-9.749279e-01


In [0]:
df_variation = df
for col in df_variation.columns:
  for i in range(len(df_variation)):
    df_variation[col][i] = (df[col][i] - df[col][i-1]) / df[col][i-1]

df_variation.head()

    #       df_variation[col][i] = 0
    # if i > 0:
    #   if df[col][i-1] > 0 :
    #     print(i)
    #     df_variation[col][i] = (df[col][i] - df[col][i-1]) / df[col][i-1]

,GOLD_USD_AM,GOLD_USD_PM,GOLD_GBP_AM,GOLD_GBP_PM,GOLD_EUR_AM,GOLD_EUR_PM,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap,ADA_Open,ADA_High,ADA_Low,ADA_Close,ADA_Volume,ADA_Market_Cap,LTC_Open,LTC_High,LTC_Low,LTC_Close,LTC_Volume,LTC_Market_Cap,EOS_Open,EOS_High,EOS_Low,EOS_Close,EOS_Volume,EOS_Market_Cap,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Volume,ETH_Market_Cap,BCH_Open,BCH_High,BCH_Low,BCH_Close,...,ICE_Low,ICE_Close,ICE_Volume,IEF_Open,IEF_High,IEF_Low,IEF_Close,IEF_Volume,NVIDIA_Open,NVIDIA_High,NVIDIA_Low,NVIDIA_Close,NVIDIA_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Volume,GOOGLE_Open,GOOGLE_High,GOOGLE_Low,GOOGLE_Close,GOOGLE_Volume,GPS_Open,GPS_High,GPS_Low,GPS_Close,GPS_Volume,COCOA_Open,COCOA_High,COCOA_Low,COCOA_Close,COCOA_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Volume,Day_Cos,Day_Sin
GOLD_Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,-0.085285,-0.087478,0.100551,0.094310,0.114992,0.108620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.663968,17.990733,17.761206,18.021827,-2.013534e-01,inf,inf,inf,inf,inf,2.505178,2.442421,2.502939,2.444330,-8.712515e-01,-0.459196,-0.45977,-0.458910,-0.463452,-1.0,NaN,NaN,NaN,NaN,NaN,-0.352594,-0.346824,-0.350275,-0.347012,-1.0,-0.753020,1.246980e+00
2019-12-30,-17723.869718,-17319.732877,11459.514653,12220.076969,11740.881765,12445.187810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.351474,12.128981,12.010941,11.891035,-3.110497e+07,NaN,NaN,NaN,NaN,NaN,537.883805,552.958665,531.981492,545.628331,-1.201796e+06,-39.785172,-40.30225,-39.395292,-39.429038,-1.0,NaN,NaN,NaN,NaN,NaN,-384.586162,-391.047451,-379.274510,-383.637646,-1.0,-1.827985,-3.730198e-01
2019-12-29,-1.085230,-1.087270,-0.899092,-0.905674,-0.884475,-0.891215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.00000,-1.000000,-1.000000,-1.0,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.547050,-1.000000e+00
2019-12-28,-0.484795,-0.485521,-0.528352,-0.533816,-0.424536,-0.429914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,-430.850261,2.719151e+18
2019-12-27,-3116.955154,-3114.152465,-2189.616609,-2160.303310,-3195.949019,-3150.121451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.999484,-1.000000e+00


In [30]:
df_variation.head(50)

,GOLD_USD_AM,GOLD_USD_PM,GOLD_GBP_AM,GOLD_GBP_PM,GOLD_EUR_AM,GOLD_EUR_PM,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap,ADA_Open,ADA_High,ADA_Low,ADA_Close,ADA_Volume,ADA_Market_Cap,LTC_Open,LTC_High,LTC_Low,LTC_Close,LTC_Volume,LTC_Market_Cap,EOS_Open,EOS_High,EOS_Low,EOS_Close,EOS_Volume,EOS_Market_Cap,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Volume,ETH_Market_Cap,BCH_Open,BCH_High,BCH_Low,BCH_Close,...,ICE_Low,ICE_Close,ICE_Volume,IEF_Open,IEF_High,IEF_Low,IEF_Close,IEF_Volume,NVIDIA_Open,NVIDIA_High,NVIDIA_Low,NVIDIA_Close,NVIDIA_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Volume,GOOGLE_Open,GOOGLE_High,GOOGLE_Low,GOOGLE_Close,GOOGLE_Volume,GPS_Open,GPS_High,GPS_Low,GPS_Close,GPS_Volume,COCOA_Open,COCOA_High,COCOA_Low,COCOA_Close,COCOA_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Volume,Day_Cos,Day_Sin
GOLD_Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,-0.085285,-0.087478,0.100551,0.094310,0.114992,0.108620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.663968,17.990733,17.761206,18.021827,-2.013534e-01,inf,inf,inf,inf,inf,2.505178,2.442421,2.502939,2.444330,-8.712515e-01,-0.459196,-0.45977,-0.458910,-0.463452,-1.0,NaN,NaN,NaN,NaN,NaN,-0.352594,-0.346824,-0.350275,-0.347012,-1.0,-0.753020,1.246980e+00
2019-12-30,-17723.869718,-17319.732877,11459.514653,12220.076969,11740.881765,12445.187810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.351474,12.128981,12.010941,11.891035,-3.110497e+07,NaN,NaN,NaN,NaN,NaN,537.883805,552.958665,531.981492,545.628331,-1.201796e+06,-39.785172,-40.30225,-39.395292,-39.429038,-1.0,NaN,NaN,NaN,NaN,NaN,-384.586162,-391.047451,-379.274510,-383.637646,-1.0,-1.827985,-3.730198e-01
2019-12-29,-1.085230,-1.087270,-0.899092,-0.905674,-0.884475,-0.891215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.00000,-1.000000,-1.000000,-1.0,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.547050,-1.000000e+00
2019-12-28,-0.484795,-0.485521,-0.528352,-0.533816,-0.424536,-0.429914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,-inf,-inf,-430.850261,2.719151e+18
2019-12-27,-3116.955154,-3114.152465,-2189.616609,-2160.303310,-3195.949019,-3150.121451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.999484,-1.000000e+00
2019-12-26,-1.478351,-1.478782,-1.526384,-1.533527,-1.421139,-1.427266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.098566,-5.661163e-01
2019-12-25,-1009.555852,-1009.262237,-756.104683,-752.587713,-948.085146,-944.019688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,